In [8]:

import json
import pandas as pd
import os
os. chdir('/Users/au324704/Desktop/AOB_corpus/')

In [9]:
# test open one file
df = pd.read_json(path_or_buf="", lines=True)

/var/folders/8t/vv7n69jj4sn7sy038gdzcqh5zptf_4/T/ipykernel_67535/1408661093.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(path_or_buf="", lines=True)


In [10]:
file_path = "data/" # set to:: data/non_english/ – if treating other languages

big_df = pd.DataFrame()

for file in os.listdir("data/"):
    if file.endswith(".json") or file.endswith(".jsonl"):

        data = pd.read_json(path_or_buf=f"{file_path}/{file}", lines=True)
        big_df = pd.concat([big_df, data], axis=0)
        print(len(big_df))


547
933
1723
2185
2694
3818
5102
5734
6281
6846


In [12]:
import json
from pathlib import Path

from lexical_diversity import lex_div as ld

import neurokit2 as nk
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
import pandas as pd
import spacy
from tqdm import tqdm

from get_metrics.utils import *

In [6]:
# if making changes to utils meanwhile, uncomment this and run

# from importlib import reload
# import utils
# reload(utils)
# from utils import *

In [13]:
# CHANGE THESE TO YOUR PREFERENCES

out_dir = "output/english"
language = "english"
#sentiment_method = "vader" # not doing SA right now, but can add later

In [14]:
# ENGLISH

# get it
if language == "english":
    spacy_language = "en_core_web_sm"
    !python3 -m spacy download en_core_web_sm
if language == 'german':
    spacy_language = "de_core_news_sm"
    !python3 -m spacy download de_core_news_sm
if language == 'italian':
    spacy_language = "it_core_news_sm"
    !python3 -m spacy download it_core_news_sm
if language == 'korean':
    spacy_language = "ko_core_news_sm"
    !python3 -m spacy download ko_core_news_sm
else:
    print("Please specify/add a language")

# load it
nlp = spacy.load(spacy_language)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Please specify/add a language


In [12]:
nlp.max_length = 3500000

nltk.download("punkt")
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/ucloud/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
len(big_df['story_id'])

6846

In [14]:
print(len(big_df))
print(len(big_df.drop_duplicates(subset='story_id')))

6846
6714


In [15]:
## get the texts to dict
text_dict = {}

for i, row in big_df.iterrows():
    content = row['story_content']
    if len(content) < 1:
        print('VERY SMALL')

    ids = row['story_id']
    if ids < 0:
        print('no id')

    text_dict[ids] = content

# we are dropping duplicates ids here. So that means the data is treating only unique ids

In [16]:
len(text_dict)

6714

In [17]:
from importlib import reload
import utils
reload(utils)
from utils import *


In [ ]:

# Initialize variables
lang = 'english'
master_dict = {}
c = 0

# Ensure out_dir is defined
out_dir = Path('./output')  # Define your output directory
out_dir.mkdir(parents=True, exist_ok=True)

# Processing loop
for key, text in tqdm(text_dict.items(), total=len(text_dict)):
    temp = {}
    file_id = str(key)
    filename = Path(file_id)
    cleaned_text = text.replace('\n', ' ')

    # Tokenization
    try:
        sents = sent_tokenize(cleaned_text, language=lang)
        words = word_tokenize(cleaned_text, language=lang)
    except Exception as e:
        print(f"Error in tokenization for {filename.name}: {e}")
        continue

    # Spacy attributes
    try:
        spacy_attributes = [get_spacy_attributes(token) for token in nlp(cleaned_text)]
        spacy_df = create_spacy_df(spacy_attributes)
        save_spacy_df(spacy_df, filename, out_dir)
    except Exception as e:
        print(f"Error in Spacy processing for {filename.name}: {e}")
        continue

    # Stylometric calculations
    temp["word_count"] = len(words)
    temp["average_wordlen"] = avg_wordlen(words)
    temp["msttr"] = ld.msttr(words, window_length=100)
    temp["average_sentlen"] = avg_sentlen(sents)

    print(file_id)
    # Skip short texts
    if len(sents) < 300:
        print(f"{file_id} has fewer than 300 sentences. Skipping advanced metrics.")
        print(len(sents))
        continue

    try:
        temp["gzipr"], temp["bzipr"] = compressrat(sents)
    except Exception as e:
        print(f"Error in compression ratios for {filename.name}: {e}")

    # Readability metrics
    try:
        temp["flesch_grade"], temp["flesch_ease"], temp["smog"], temp["ari"], temp['dale_chall'] = text_readability(cleaned_text, lang)
    except Exception as e:
        print(f"Error in readability metrics for {filename.name}: {e}")

    # Save results
    master_dict[filename.stem] = temp
    c += 1

    # Save intermediate results
    if c % 100 == 0:
        with open(f'master_dict_{lang}_checkpoint.json', 'w') as f:
            json.dump(master_dict, f)

# Final save
with open(f'master_dict_{lang}.json', 'w') as f:
    json.dump(master_dict, f)

  0%|          | 0/6714 [00:00<?, ?it/s]

  0%|          | 1/6714 [00:00<57:45,  1.94it/s]

10718370


  0%|          | 2/6714 [00:00<37:55,  2.95it/s]

10853679
10853679 has fewer than 300 sentences. Skipping advanced metrics.
57


...

41328993
41328993 has fewer than 300 sentences. Skipping advanced metrics.
50


 42%|████▏     | 2852/6714 [37:33<47:20,  1.36it/s]

41375343
41375343 has fewer than 300 sentences. Skipping advanced metrics.
129


 42%|████▏     | 2853/6714 [37:33<40:39,  1.58it/s]

4140795
4140795 has fewer than 300 sentences. Skipping advanced metrics.
127


In [ ]:
# check spacy books made
lang = 'english' 

n_spacy_books = []
for file in os.listdir(f"output/{lang}/spacy_books"):
    if file.endswith(".csv"):
        print('filename treated:', file)
        n_spacy_books.append(file)

print('len textdict, n spacy books:', len(text_dict), len(n_spacy_books))


filename treated: 30817547_spacy.csv
filename treated: 30836849_spacy.csv
filename treated: 40495320_spacy.csv
filename treated: 5815870_spacy.csv
filename treated: 34303951_spacy.csv
filename treated: 28429236_spacy.csv
filename treated: 30383361_spacy.csv
filename treated: 8094787_spacy.csv
filename treated: 17325110_spacy.csv
filename treated: 4421315_spacy.csv
filename treated: 13746708_spacy.csv
filename treated: 34197379_spacy.csv
filename treated: 13606332_spacy.csv
filename treated: 32986963_spacy.csv
filename treated: 22666765_spacy.csv
filename treated: 17005434_spacy.csv
filename treated: 5602843_spacy.csv
filename treated: 23989756_spacy.csv
filename treated: 22318132_spacy.csv
filename treated: 25775014_spacy.csv
filename treated: 17197178_spacy.csv
filename treated: 34702753_spacy.csv
filename treated: 35998432_spacy.csv
filename treated: 11215386_spacy.csv
filename treated: 31430243_spacy.csv
filename treated: 27699850_spacy.csv
filename treated: 30137919_spacy.csv
filen

In [ ]:
# %%
lang = 'english' # NB!

spacy_dict = {}

for file in os.listdir(f"output/spacy_books"):
    if file.endswith(".csv"):
        temp = {}

        data = pd.read_csv(f"output/spacy_books/{file}")

        new_list = data['token_pos_'].to_list()
        n_noun = new_list.count('NOUN')
        n_verb = new_list.count('VERB')
        n_adj = new_list.count('ADJ')

        temp['nominal_ratio'] = (n_noun + n_adj) / n_verb
        temp['nominal_ratio_inverse'] = n_verb / (n_noun + n_adj)


        if len(data)>40:
            full_stop_indices = data[data['token_text'].str.strip() == '.'].index
            
            # Adding the last index of the DataFrame to handle the last sentence
            full_stop_indices = list(full_stop_indices) + [data.index[-1]]
        
            temp['NDD_mean'], temp['NDD_std'], temp['DD_mean'], temp['DD_std'] = calculate_dependency_distances(data, full_stop_indices)

        # and TTRs
            split = data.loc[10:1010]
            
            # Get TTR of verbs
            df_verbs = split.loc[split['token_pos_'] == 'VERB']
            no_verbs = len(df_verbs)
            raw_verb_tokens = list(df_verbs['token_text'].str.lower().astype(str))
            no_types_verbs = len(set(raw_verb_tokens))
            # print(no_verbs, no_types_verbs)
            temp['TTR_VERB'] = no_types_verbs/no_verbs

            # Get TTR of nouns
            df_nouns = split.loc[split['token_pos_'] == 'NOUN']
            no_nouns = len(df_nouns)
            raw_noun_tokens = list(df_nouns['token_text'].str.lower().astype(str))
            no_types_nouns = len(set(raw_noun_tokens))
            #print(no_nouns, no_types_nouns)
            temp['TTR_NOUN'] = no_types_nouns/no_nouns


        filename = file.split('.')[0].split('_')[0]
        spacy_dict[filename] = temp

with open(f'spacy_master_dict_{lang}.json', 'w') as f:
    json.dump(spacy_dict, f)

In [ ]:
# we want to open and merge the data

In [ ]:
with open(f'master_dict_{lang}.json', 'r') as f:
    res = json.load(f)

res_df = pd.DataFrame.from_dict(res, orient='index').reset_index()
res_df.rename(columns={"index": "STORY_ID"}, inplace=True)
res_df

,STORY_ID,word_count,average_wordlen,msttr,average_sentlen,gzipr,bzipr
0,29559000,19170,4.150652,0.732984,106.164983,2.651235,3.238676
1,30137445,3701,4.319914,0.749459,70.473881,NaN,NaN
2,30137919,4572,4.279965,0.743778,72.691824,2.255266,2.472309
3,30363438,1868,4.475910,0.784444,201.612245,NaN,NaN
4,30383361,3913,4.278048,0.754359,190.730769,NaN,NaN
...,...,...,...,...,...,...,...
192,40391016,86598,4.308033,0.744578,384.465278,2.977572,4.141186
193,40495320,4522,4.305617,0.772889,94.428571,NaN,NaN
194,40742178,13308,4.025699,0.743684,120.119403,2.849559,3.427737
195,40891257,7785,4.370071,0.743247,81.706967,2.599486,3.063778


In [ ]:
with open(f'spacy_master_dict_{lang}.json', 'r') as f:
    synt = json.load(f)

synt_df = pd.DataFrame.from_dict(synt, orient='index').reset_index()
synt_df.rename(columns={"index": "STORY_ID"}, inplace=True)
synt_df

,STORY_ID,nominal_ratio,nominal_ratio_inverse,NDD_mean,NDD_std,DD_mean,DD_std,TTR_VERB,TTR_NOUN
0,30817547,1.290938,0.774631,1.271959,0.475414,2.152662,0.709948,0.827338,0.711409
1,30836849,1.657895,0.603175,1.390944,0.520307,2.187319,0.888392,0.966942,0.812500
2,40495320,1.792321,0.557936,1.345561,0.463296,2.197680,0.615066,0.936000,0.911765
3,5815870,1.195890,0.836198,1.169549,0.378565,1.821320,0.631245,0.875000,0.750000
4,34303951,1.544304,0.647541,1.269146,0.530597,2.400199,0.648035,0.948718,0.853659
...,...,...,...,...,...,...,...,...,...
192,26752276,1.687919,0.592445,1.345857,0.520546,2.448002,0.783682,0.870229,0.736527
193,13632471,0.961022,1.040559,1.117381,0.444403,1.977050,0.642359,0.809524,0.804124
194,34642039,1.418182,0.705128,1.105689,0.449400,1.899106,0.811355,0.836364,0.750000
195,30423477,1.704545,0.586667,1.163592,0.543175,2.134487,0.961427,0.935484,0.860140


In [ ]:
# merge
merged = res_df.merge(synt_df, on='STORY_ID')
merged.columns = [x.lower() for x in merged.columns]
merged

,story_id,word_count,average_wordlen,msttr,average_sentlen,gzipr,bzipr,nominal_ratio,nominal_ratio_inverse,ndd_mean,ndd_std,dd_mean,dd_std,ttr_verb,ttr_noun
0,29559000,19170,4.150652,0.732984,106.164983,2.651235,3.238676,1.534751,0.651572,1.221525,0.436040,2.089507,0.752621,0.952381,0.872483
1,30137445,3701,4.319914,0.749459,70.473881,NaN,NaN,1.449020,0.690122,1.234201,0.451590,1.845514,0.667904,0.919643,0.845679
2,30137919,4572,4.279965,0.743778,72.691824,2.255266,2.472309,1.847036,0.541408,1.214905,0.465770,2.124499,0.774331,0.931373,0.899371
3,30363438,1868,4.475910,0.784444,201.612245,NaN,NaN,2.045455,0.488889,1.471727,0.526648,2.485873,0.701959,0.926606,0.829114
4,30383361,3913,4.278048,0.754359,190.730769,NaN,NaN,1.576200,0.634437,1.368382,0.484224,2.430864,0.633823,0.990000,0.921053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,40391016,86598,4.308033,0.744578,384.465278,2.977572,4.141186,1.246118,0.802492,1.795982,0.568970,2.963554,1.000976,0.863636,0.702532
193,40495320,4522,4.305617,0.772889,94.428571,NaN,NaN,1.792321,0.557936,1.345561,0.463296,2.197680,0.615066,0.936000,0.911765
194,40742178,13308,4.025699,0.743684,120.119403,2.849559,3.427737,1.406426,0.711022,1.334115,0.448260,2.118506,0.740293,0.858407,0.805195
195,40891257,7785,4.370071,0.743247,81.706967,2.599486,3.063778,1.492895,0.669839,1.258380,0.424703,1.882573,0.672234,0.918033,0.637755


In [ ]:
# save as json
merged.to_json(f'{lang}_ABO_metrics.json', orient = 'split')